# Tensorflow Estimators

In [1]:
import tensorflow as tf
import pandas as pd

In [2]:
df = pd.read_csv('../data/iris.csv')

In [3]:
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [4]:
X = df.drop('species', axis='columns')
y = df['species']

column_names = df.columns[:-1]

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [7]:
y_train.head()

60     versicolor
116     virginica
144     virginica
119     virginica
108     virginica
Name: species, dtype: object

In [8]:
labels = ['setosa', 'versicolor', 'virginica']

In [9]:
feature_columns = [
    tf.feature_column.numeric_column(key=key)
    for key in X_train.columns
]

In [10]:
feature_columns

[_NumericColumn(key='sepal_length', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='sepal_width', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='petal_length', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='petal_width', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

## Canned Estimators

In [11]:
rm -rf ./models/iris/

In [12]:
model = tf.estimator.DNNClassifier(
    hidden_units=[10, 10], 
    feature_columns=feature_columns,
    model_dir='./models/iris/DNN/',
    n_classes=3,
    label_vocabulary=labels,
    optimizer='Adam')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './models/iris/DNN/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f92c7046320>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [13]:
batch_size=32
train_steps=3000

### Input function

In [14]:
def input_fn(features, labels, batch_size, shuffle=True, repeat=None):
    feature_dict = dict(features)

    if labels is None:
        inputs = feature_dict
    else:
        inputs = (feature_dict, labels)
        
    dataset = tf.data.Dataset.from_tensor_slices(inputs)
    
    if shuffle:
        dataset = dataset.shuffle(1000)
    
    dataset = dataset.repeat(repeat)

    return dataset.batch(batch_size)

In [15]:
def train_input_fn():
    return input_fn(X_train, y_train, batch_size)

### Model Training

In [16]:
model.train(
    input_fn=train_input_fn,
    steps=3000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into ./models/iris/DNN/model.ckpt.
INFO:tensorflow:loss = 37.27954, step = 0
INFO:tensorflow:global_step/sec: 256.354
INFO:tensorflow:loss = 1.8347392, step = 100 (0.392 sec)
INFO:tensorflow:global_step/sec: 257.756
INFO:tensorflow:loss = 1.8607783, step = 200 (0.388 sec)
INFO:tensorflow:global_step/sec: 281.685
INFO:tensorflow:loss = 1.1390328, step = 300 (0.355 sec)
INFO:tensorflow:global_step/sec: 315.224
INFO:tensorflow:loss = 0.16796286, step = 400 (0.317 sec)
INFO:tensorflow:global_step/sec: 307.916
INFO:tensorflow:loss = 0.16669615, step = 500 (0.325 sec)
INFO:tensorflow:global_step/sec: 288.356
INFO:tensorflow:loss = 0.43382812, step = 600 (0.347 sec)
INFO:tensorflow:global_step/sec: 302.036
INFO:tensorfl

### Evaluate model on train and test data

In [17]:
train_eval_result = model.evaluate(
    input_fn=lambda:input_fn(X_train, y_train,
                             batch_size,
                             shuffle=False,
                             repeat=1))

test_eval_result = model.evaluate(
    input_fn=lambda:input_fn(X_test, y_test,
                             batch_size,
                             shuffle=False,
                             repeat=1))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-09-21-17:12:24
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./models/iris/DNN/model.ckpt-3000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-09-21-17:12:24
INFO:tensorflow:Saving dict for global step 3000: accuracy = 0.9809524, average_loss = 0.029021518, global_step = 3000, loss = 0.76181483
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 3000: ./models/iris/DNN/model.ckpt-3000
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-09-21-17:12:25
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./models/iris/DNN/model.ckpt-3000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 201

In [18]:
train_eval_result

{'accuracy': 0.9809524,
 'average_loss': 0.029021518,
 'loss': 0.76181483,
 'global_step': 3000}

In [19]:
test_eval_result

{'accuracy': 0.9777778,
 'average_loss': 0.11544406,
 'loss': 2.5974913,
 'global_step': 3000}

### Predict on new data

In [20]:
predict_x = {
    'sepal_length': [5.1, 5.9, 6.9],
    'sepal_width': [3.3, 3.0, 3.1],
    'petal_length': [1.7, 4.2, 5.4],
    'petal_width': [0.5, 1.5, 2.1],
}

In [21]:
expected = ['setosa', 'versicolor', 'virginica']

In [22]:
predictions = model.predict(
    input_fn=lambda:input_fn(predict_x,
                             labels=None,
                             batch_size=batch_size,
                             shuffle=False,
                             repeat=1))

In [23]:
predictions

<generator object Estimator.predict at 0x7f929d669200>

In [24]:
template = ('\nPrediction is "{}", prob: {:.1f}%, expected "{}"')

for pred_dict, expec in zip(predictions, expected):
    
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]
    pred_label = labels[class_id]

    print(template.format(pred_label, 100 * probability, expec))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./models/iris/DNN/model.ckpt-3000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.

Prediction is "setosa", prob: 100.0%, expected "setosa"

Prediction is "versicolor", prob: 99.9%, expected "versicolor"

Prediction is "virginica", prob: 99.7%, expected "virginica"


In [25]:
pred_dict

{'logits': array([-14.253295 ,  -2.5915875,   3.1705759], dtype=float32),
 'probabilities': array([2.7011327e-08, 3.1344465e-03, 9.9686545e-01], dtype=float32),
 'class_ids': array([2]),
 'classes': array([b'virginica'], dtype=object)}

## Exercise 1

The `tf.estimator` module offers 2 additional canned classifiers:

- `tf.estimator.BaselineClassifier`
- `tf.estimator.LinearClassifier`

What do they do?

Try substituting them in the model definition and re-run the trainin, evaluation and prediction steps.
Is the performance better or worse?

Note: you can set the logging level of tensorflow with `tf.logging.set_verbosity`

In [26]:
tf.logging.set_verbosity(tf.logging.WARN)

#### Baseline Classifier

In [27]:
model = tf.estimator.BaselineClassifier(
    model_dir='./models/iris/Baseline/',
    n_classes=3,
    label_vocabulary=labels)

In [28]:
model.train(
    input_fn=train_input_fn,
    steps=3000)

In [29]:
train_eval_result = model.evaluate(
    input_fn=lambda:input_fn(X_train, y_train,
                             batch_size,
                             shuffle=False,
                             repeat=1))

test_eval_result = model.evaluate(
    input_fn=lambda:input_fn(X_test, y_test,
                             batch_size,
                             shuffle=False,
                             repeat=1))

In [30]:
train_eval_result

{'accuracy': 0.37142858,
 'average_loss': 1.0951546,
 'loss': 28.747808,
 'global_step': 3000}

In [31]:
test_eval_result

{'accuracy': 0.24444444,
 'average_loss': 1.117924,
 'loss': 25.15329,
 'global_step': 3000}

#### Linear Classifier

In [32]:
model = tf.estimator.LinearClassifier(
    feature_columns=feature_columns,
    model_dir='./models/iris/Linear/',
    n_classes=3,
    label_vocabulary=labels)

In [33]:
model.train(
    input_fn=train_input_fn,
    steps=3000)

In [34]:
train_eval_result = model.evaluate(
    input_fn=lambda:input_fn(X_train, y_train,
                             batch_size,
                             shuffle=False,
                             repeat=1))

test_eval_result = model.evaluate(
    input_fn=lambda:input_fn(X_test, y_test,
                             batch_size,
                             shuffle=False,
                             repeat=1))

In [35]:
train_eval_result

{'accuracy': 0.9809524,
 'average_loss': 0.08735905,
 'loss': 2.293175,
 'global_step': 3000}

In [36]:
test_eval_result

{'accuracy': 0.9777778,
 'average_loss': 0.11286557,
 'loss': 2.5394752,
 'global_step': 3000}

## Exercise 2

Visualize the training results with tensorboard:

- in a terminal, from the notebook folder, run tensorboard with `tensorboard --logdir=models`
- open the browser at [http://localhost:6006](http://localhost:6006)

